> 08 슈팅
- 좌우 키로 내 캐릭터를 이동하고 스페이스 키로 미사일을 발사합니다 .다가오는 외계인을 모두 쏘세요. 바리케이트가 없고 UFO가 출현하지 않는 등 없는것 투성이지만 그래도 한 시대를 풍미했던 게임의 분위기는 엿볼 수 있습니다

In [6]:
#### invader ####
import sys
from random import randint
import pygame
from pygame.locals import QUIT, KEYDOWN, Rect, K_LEFT, K_RIGHT, K_SPACE

pygame.init()
pygame.key.set_repeat(5, 5)
SURFACE = pygame.display.set_mode((600,600))
FPSCLOCK = pygame.time.Clock()

class Drawable: #전체의 그리기 객체의 슈퍼클래스
    def __init__(self,rect,offset0,offset1):
        strip = pygame.image.load('strip.png')
        self.images = (pygame.Surface((24,24),pygame.SRCALPHA),pygame.Surface((24,24),pygame.SRCALPHA)) #그리는이미지의배열
        self.rect = rect    #그리는 위치와 크기
        self.count = 0  #그리는 이미지를 전환하기 위한 카운터
        self.images[0].blit(strip,(0,0),Rect(offset0,0,24,24))
        self.images[1].blit(strip,(0,0),Rect(offset1,0,24,24))
    def move(self,diff_x,diff_y):
        self.count += 1
        self.rect.move_ip(diff_x,diff_y)

    def draw(self):
        image = self.images[0] if self.count % 2 == 0 else self.images[1]
        SURFACE.blit(image,self.rect.topleft)
class Ship(Drawable): #우주선 클래스
    def __init__(self):
        super().__init__(Rect(300,550,24,24),192,192)   #192,192은 우주선 이미지의 좌표

class Beam(Drawable): #빔 클래스
    def __init__(self):
        super().__init__(Rect(300,0,24,24),0,24)   #24,24은 빔 이미지의 좌표

class Bomb(Drawable): #폭탄 클래스
    def __init__(self):
        super().__init__(Rect(300,-50,24,24),48,72)   #48,48은 폭탄 이미지의 좌표
        self.time = randint(5,220)  #폭탄이 떨어지기까지의 시간

class Alien(Drawable): #외계인 클래스
    def __init__(self,rect,offset,score):
        super().__init__(rect,offset,offset+24)   #외계인 이미지의 좌표
        self.score = score  #이 외계인을 쓰러뜨렸을 때의 점수

def main():
    sysfont = pygame.font.SysFont(None, 48)
    scorefont = pygame.font.SysFont(None, 36)
    message_clear = sysfont.render('CLEARED!', True, (255,255,255))
    message_over = sysfont.render('GAME OVER', True, (255,255,255))
    message_rect=message_clear.get_rect()
    message_rect.center = (300,300)
    game_over = False
    moving_left =True   #외계인 전체가 왼쪽 방향으로 움직이고 있는지 아닌지
    moving_down = False#외계인 전체가 아래 방향으로 움직이고 있는지 아닌지
    move_interval = 20  #외계인이 이동할 때가지의 프레임수
    counter = 0 #시간관리용 카운터
    score = 0
    aliens = []
    bombs=[]
    ship=Ship()
    beam=Beam()

    #외계인 나열을 초기화
    for ypos in range(4):
        offset = 96 if ypos <2 else 144 #외계인 이미지의 y좌표로 96과 144는 다른 종족의 얼라인이다.
        for xpos in range(10):
            rect= Rect(100+xpos*50, ypos*50+50,24,24)
            alien = Alien(rect,offset,(4-ypos)*10)
            aliens.append(alien)
    #폭탄을 설정
    for __ in range(4):
        bombs.append(Bomb())
    while True:
        ship_move_x = 0
        for event in pygame.event.get():
            if event.type == QUIT:
                pygame.quit()
                sys.exit()
            elif event.type == KEYDOWN:
                if event.key == K_LEFT:
                    ship_move_x = -5
                elif event.key == K_RIGHT:
                    ship_move_x = 5
                elif event.key == K_SPACE and beam.rect.bottom < 0:
                    beam.rect.center = ship.rect.center #빔 위치를 내 캐릭터 위치로 초기화한다.

            

        if not game_over:
            counter += 1
            #내 케릭터를이동
            ship.move(ship_move_x,0)    
            #빔을이동
            
            
            beam.move(0,-48)
                       #외계인을이동
            area=aliens[0].rect.copy()  #리스트의 맨 앞의외계인직사각형을 복사해서 area변수에 저장한다
            for alien in aliens:
                area.union_ip(alien.rect)   #union_ip()는 인수의 직사각형을 포함하도록 area를 확장한다. 즉 모든왹몌인을 호출함으로서 모든외계인을 포함하는 최소 직사각형을 구합니다.
            if counter % move_interval ==0: #일정프레임마다외계인전체를이동 초기값은 20으로 설정되었으므로 20프레임마다이동
                move_x= -5 if moving_left else 5
                move_y=0
                if(area.left <10 or area.right >590) and not moving_down:
                    moving_left = not moving_left   #좌우 이동방향을 반전한다
                    move_x,move_y = 0,24    #아래방향의 이동량
                    move_interval =max(1,move_interval -2)  #외계인의 이동속도를 올린다
                    moving_down = True  #전체외계인에게 그 다음 이동은 아래방향임을 알린다
                else:
                    moving_down = False
                for alien in aliens:
                    alien.move(move_x,move_y)
            if area.bottom > 550:
                game_over = True
                   
            #폭탄을이동
            for bomb in bombs:
                if bomb.time < counter and bomb.rect.top < 0:   #폭탄의 대시기간이 지나고 폭탄이 떨어지고 있지않은경우 투하시작한다.
                    enemy = aliens[randint(0,len(aliens)-1)]    #폭탄을 떨어뜨릴 외계인을 랜덤하게 선택
                    bomb.rect.center=enemy.rect.center  #폭탄의 위치를 외계인의 위치로 초기화

                if bomb.rect.top> 0:    #랜점으로 설정되면 폭탄이 낙하되기 시작한다. 10의 속도로
                    bomb.move(0,10)
                if bomb.rect.top > 600: #맨 아래에 도착하면
                    bomb.rect.top = -50 #폭탄을 다시 위로 올린다 화면밖으로 배치
                    bomb.time += randint(50,250)
                if bomb.rect.colliderect(ship.rect): #폭탄과 우주선 충돌?
                    game_over = True
            #빔과 외계인 충돌?
            tmp=[]
            for alien in aliens:
                if alien.rect.colliderect(beam.rect):
                    score += alien.score
                    beam.rect.top = -50
                else:
                    tmp.append(alien)
            aliens = tmp
            if len(aliens) ==0:
                game_over = True
        SURFACE.fill((0, 0, 0))
        for alien in aliens:
            alien.draw()
        ship.draw()
        beam.draw()
        for bomb in bombs:
            bomb.draw()
        score_str=str(score).zfill(5)
        score_image=scorefont.render(score_str,True,(255,255,255))
        SURFACE.blit(score_image,(10,10))
        if game_over:
            if len(aliens) ==0:
                SURFACE.blit(message_clear,message_rect)
            else:   
                SURFACE.blit(message_over,message_rect)
        pygame.display.update()
        FPSCLOCK.tick(20)
if __name__ == '__main__':
    main()

SystemExit: 

In [1]:
""" invader.py - Copyright 2016 Kenichiro Tanaka  """
import sys
from random import randint
import pygame
from pygame.locals import Rect, QUIT, KEYDOWN, \
    K_LEFT, K_RIGHT, K_SPACE

pygame.init()
pygame.key.set_repeat(5, 5)
SURFACE = pygame.display.set_mode((600, 600))
FPSCLOCK = pygame.time.Clock()

class Drawable:
    """ 전체의 그리기 객체의 슈퍼 클래스 """
    def __init__(self, rect, offset0, offset1):
        strip = pygame.image.load("strip.png")
        self.images = (pygame.Surface((24, 24), pygame.SRCALPHA),
                       pygame.Surface((24, 24), pygame.SRCALPHA))
        self.rect = rect
        self.count = 0
        self.images[0].blit(strip, (0, 0),
                            Rect(offset0, 0, 24, 24))
        self.images[1].blit(strip, (0, 0),
                            Rect(offset1, 0, 24, 24))

    def move(self, diff_x, diff_y):
        """ 객체를 이동 """
        self.count += 1
        self.rect.move_ip(diff_x, diff_y)

    def draw(self):
        """ 객체를 그리기 """
        image = self.images[0] if self.count % 2 == 0 \
                else self.images[1]
        SURFACE.blit(image, self.rect.topleft)

class Ship(Drawable):
    """ 내 캐릭터 객체 """
    def __init__(self):
        super().__init__(Rect(300, 550, 24, 24), 192, 192)

class Beam(Drawable):
    """ 빔 객체 """
    def __init__(self):
        super().__init__(Rect(300, 0, 24, 24), 0, 24)

class Bomb(Drawable):
    """ 폭탄 객체 """
    def __init__(self):
        super().__init__(Rect(300, -50, 24, 24), 48, 72)
        self.time = randint(5, 220)

class Alien(Drawable):
    """ 외계인 객체 """
    def __init__(self, rect, offset, score):
        super().__init__(rect, offset, offset+24)
        self.score = score

def main():
    """ 메인 루틴 """
    sysfont = pygame.font.SysFont(None, 72)
    scorefont = pygame.font.SysFont(None, 36)
    message_clear = sysfont.render("!!CLEARED!!",
                                   True, (0, 255, 225))
    message_over = sysfont.render("GAME OVER!!",
                                  True, (0, 255, 225))
    message_rect = message_clear.get_rect()
    message_rect.center = (300, 300)
    game_over = False
    moving_left = True
    moving_down = False
    move_interval = 20
    counter = 0
    score = 0
    aliens = []
    bombs = []
    ship = Ship()
    beam = Beam()

    # 외계인 나열과 초기화
    for ypos in range(4):
        offset = 96 if ypos < 2 else 144
        for xpos in range(10):
            rect = Rect(100+xpos*50, ypos*50 + 50, 24, 24)
            alien = Alien(rect, offset, (4-ypos)*10)
            aliens.append(alien)

    # 폭탄을 설정
    for _ in range(4):
        bombs.append(Bomb())

    while True:
        ship_move_x = 0
        for event in pygame.event.get():
            if event.type == QUIT:
                pygame.quit()
                sys.exit()
            elif event.type == KEYDOWN:
                if event.key == K_LEFT:
                    ship_move_x = -5
                elif event.key == K_RIGHT:
                    ship_move_x = +5
                elif event.key == K_SPACE and beam.rect.bottom < 0:
                    beam.rect.center = ship.rect.center

        if not game_over:
            counter += 1
            # 내 캐릭터를 이동
            ship.move(ship_move_x, 0)

            # 빔을 이동
            beam.move(0, -15)

            # 외계인을 이동
            area = aliens[0].rect.copy()
            for alien in aliens:
                area.union_ip(alien.rect)

            if counter % move_interval == 0:
                move_x = -5 if moving_left else 5
                move_y = 0

                if (area.left < 10 or area.right > 590) and \
                    not moving_down:
                    moving_left = not moving_left
                    move_x, move_y = 0, 24
                    move_interval = max(1, move_interval - 2)
                    moving_down = True
                else:
                    moving_down = False

                for alien in aliens:
                    alien.move(move_x, move_y)

            if area.bottom > 550:
                game_over = True

            # 폭탄을 이동
            for bomb in bombs:
                if bomb.time < counter and bomb.rect.top < 0:
                    enemy = aliens[randint(0, len(aliens) - 1)]
                    bomb.rect.center = enemy.rect.center

                if bomb.rect.top > 0:
                    bomb.move(0, 10)

                if bomb.rect.top > 600:
                    bomb.time += randint(50, 250)
                    bomb.rect.top = -50

                if bomb.rect.colliderect(ship.rect):
                    game_over = True

            # 빔과 외계인 충돌?
            tmp = []
            for alien in aliens:
                if alien.rect.collidepoint(beam.rect.center):
                    beam.rect.top = -50
                    score += alien.score
                else:
                    tmp.append(alien)
            aliens = tmp
            if len(aliens) == 0:
                game_over = True

        # 그리기
        SURFACE.fill((0, 0, 0))
        for alien in aliens:
            alien.draw()
        ship.draw()
        beam.draw()
        for bomb in bombs:
            bomb.draw()

        score_str = str(score).zfill(5)
        score_image = scorefont.render(score_str,
                                       True, (0, 255, 0))
        SURFACE.blit(score_image, (500, 10))

        if game_over:
            if len(aliens) == 0:
                SURFACE.blit(message_clear, message_rect.topleft)
            else:
                SURFACE.blit(message_over, message_rect.topleft)

        pygame.display.update()
        FPSCLOCK.tick(20)

if __name__ == '__main__':
    main()


c:\Users\고창섭\AppData\Local\Programs\Python\Python313\Lib\site-packages\pygame\pkgdata.py:25: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_stream, resource_exists


pygame 2.6.1 (SDL 2.28.4, Python 3.13.7)
Hello from the pygame community. https://www.pygame.org/contribute.html


SystemExit: 

C:\Users\고창섭\AppData\Roaming\Python\Python313\site-packages\IPython\core\interactiveshell.py:3707: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
